In [25]:
import torch
import torchvision
from lib.fcn import FCN

In [26]:
gpu_name = "cuda:0"
device = torch.device(gpu_name if torch.cuda.is_available() else "cpu")

In [27]:
transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
])
_train = torchvision.datasets.MNIST(root='data/mnist', train=True,
                                    download=True, transform=transform)
train = torch.utils.data.DataLoader(_train, batch_size=32,
                                    shuffle=True, num_workers=2)

_test = torchvision.datasets.MNIST(root='data/mnist', train=True,
                                   download=True, transform=transform)
test = torch.utils.data.DataLoader(_test, batch_size=32,
                                   shuffle=False, num_workers=2)

In [28]:
model = FCN()
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [29]:
total = 0
correct = 0
for epoch in range(5):
    # train model
    model.train()
    for data in train:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # eval model
    model.eval()
    total = 0
    correct = 0
    for data in test:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
                
        outputs = model(inputs)
        prediction = outputs.max(1)[1]
                
        total += labels.size(0)
        correct += prediction.eq(labels).sum().item()

    test_acc = 100*correct/total
    print("[Epoch:%2d] Test: %.2f%%"%(epoch+1,test_acc))
    

[Epoch: 1] Test: 95.22%
[Epoch: 2] Test: 98.09%
[Epoch: 3] Test: 98.65%
[Epoch: 4] Test: 98.81%
[Epoch: 5] Test: 99.04%


In [30]:
import os
if not os.path.isdir("model"):
    os.makedirs("model")
torch.save(model.state_dict(), "model/FCN_MNIST_5")